<h1><center>Homemade implementations of Extension 2 - Logistic Regression with L1 regularization</center></h1>

### Imports

In [1]:
from PIL import Image
import numpy as np
import glob
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt

### Get the dogs data

I took the Chihuahua and Japanese Spaniel images and put them in the "selected_images/class0" directory and I put the Maltese and Pekinese images in the "selected_images/class1" directory.

So the classifier is distinguishing between \[Chihuahua or Japanese Spaniel\] and \[Maltese or Pekinese\].

In [2]:
def get_all_data():
    pics = list()
    y_all = np.array([], dtype=np.int8)
    new_size = (6, 6)

    for i, folder in enumerate(["selected_images/class0", "selected_images/class1"]):
        for f in glob.glob(f"{folder}/*"):
            curr_pic = np.array(Image.open(f).resize(new_size)).reshape(-1) # shrink and flatten the pic
            pics.append(curr_pic)
            y_all = np.concatenate((y_all, [i])) # all pictures in a given folder have the same label
            
    X_all = np.array(pics)
    return X_all, y_all

In [3]:
def add_ones(m):
    ones = np.ones(m.shape[0]).reshape(-1, 1)
    return np.hstack((ones, m))

In [4]:
X_all, y_all = get_all_data()
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.8, random_state=0)
X_train = StandardScaler().fit_transform(X_train)
X_train = add_ones(X_train)
X_test = StandardScaler().fit_transform(X_test)
X_test = add_ones(X_test)

### Get the breast cancer data

In [5]:
X_all_bc, y_all_bc = load_breast_cancer(return_X_y=True)
X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(X_all_bc, y_all_bc, train_size=0.8, random_state=0)
scaler = StandardScaler().fit(X_train_bc)
X_train_bc = scaler.transform(X_train_bc)
X_train_bc = add_ones(X_train_bc)
X_test_bc = scaler.transform(X_test_bc)
X_test_bc = add_ones(X_test_bc)

### Homemade implementation of logistic regression with L1 regularization

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def hypothesis(X , w):
    return sigmoid(X @ w)

classify = np.vectorize(lambda x: 1 if x >= 0.5 else 0)

In [7]:
def L1_grad(w): 
    grad = np.sign(w)
    grad[0][0] = 0     # don't penalize the intercept term
    return grad

def Logistic_Regression_Gradient_Ascent(X, y, learning_rate, reg_rate, num_iters, reg):
    log_likelihood_values = []
    w = np.zeros((X.shape[1], 1))
    N = X.shape[0]
    y = y.reshape(-1,1)
    
    for i in range(num_iters):
        h = hypothesis(X, w)
        w += (learning_rate / N) * (X.T @ (y - h))
        if reg:
            new_w = w - (learning_rate * reg_rate / N) * L1_grad(w)
            new_w[np.sign(new_w) != np.sign(w)] = 0 # clipped L1 from page 479 of the paper
            w = new_w

    return w

In [8]:
def get_LR_f1(train_size, X_all, y_all):
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=train_size, random_state=0)
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    plain_w = Logistic_Regression_Gradient_Ascent(X_train, y_train, 0.1, 0, 1000, False)
    plain_preds = classify(hypothesis(X_test, plain_w))    
    L1_w = Logistic_Regression_Gradient_Ascent(X_train, y_train, 0.1, 10, 1000, True)
    L1_preds = classify(hypothesis(X_test, L1_w))
    
    return 100 * f1_score(y_test, plain_preds), 100 * f1_score(y_test, L1_preds)
    

# Evaluate my homemade implementation's F1 scores

### i) Check the F1 score of my logistic regression with L1 regularization on the dogs data

In [9]:
plain_f1, l1_f1 = get_LR_f1(0.8, X_all, y_all)
print(f"The F1 score of my LR with L1 on the dog data is {l1_f1}")

The F1 score of my LR with L1 on the dog data is 64.86486486486486


### ii) Check the F1 score of my logistic regression with L1 regularization on the breast cancer data

In [10]:
l1_w_bc = Logistic_Regression_Gradient_Ascent(X_train_bc, y_train_bc, 0.1, 10, 1000, True)
l1_preds_bc = classify(hypothesis(X_test_bc, l1_w_bc))
print(f"The F1 score of my LR with L1 on the breast cancer data is {100 * f1_score(y_test_bc, l1_preds_bc)}")

The F1 score of my LR with L1 on the breast cancer data is 97.10144927536231
